In [1]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "ibm-granite/granite-3.0-1b-a400m-instruct"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Sauvegarder localement
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [4]:
train_data = [
    {"input": "What is Core Capital's main goal?", "output": "Core Capital aims to help businesses monitor and manage their financial health effectively."},
    {"input": "What financial metrics does the dashboard display?", "output": "The dashboard shows metrics like cash flow, revenue, expenses, and profit."},
    {"input": "Can I generate financial reports?", "output": "Yes, you can generate reports in PDF, Excel, or CSV formats based on custom time periods and metrics."},
    {"input": "How can I see user information?", "output": "User information is available under 'Users' but is visible only to admins."},
    {"input": "Does Core Capital provide predictive financial insights?", "output": "Yes, it analyzes past expenses to provide insights and recommendations for future planning."},
    {"input": "What recommendations can I find in notifications?", "output": "Notifications offer personalized recommendations to improve financial management."},
    {"input": "Can I edit my personal information?", "output": "Yes, click on the user avatar to view and edit your profile information."},
    {"input": "Does the system handle large amounts of data?", "output": "Yes, Core Capital is designed to be scalable and can handle large data volumes efficiently."},
    {"input": "What kind of expenses can I track?", "output": "You can log and categorize expenses in real-time and view visual spending trends."},
    {"input": "Can B4 predict metrics on the platform?", "output": "Yes, B4 can predict all financial metrics and provide useful insights."},
    {"input": "Quel est le revenu total pour 2023 ?", "output": "SELECT SUM(amount) AS total_revenue FROM revenue WHERE strftime('%Y', date) = '2023';"},
    {"input": "Quels départements ont un budget supérieur à 100000 ?", "output": "SELECT department, allocated_budget FROM budget WHERE allocated_budget > 100000;"},
    {"input": "Quelle est la dépense totale pour le département 'Marketing' ?", "output": "SELECT SUM(amount) AS total_expenses FROM expenses WHERE department = 'Marketing';"},
    {"input": "Quel est le cash flow net pour septembre 2024 ?", "output": "SELECT net_cash_flow FROM cashflow WHERE strftime('%Y-%m', date) = '2024-09';"},
    {"input": "Quelle est la balance restante de la dette ?", "output": "SELECT SUM(outstanding_balance) AS total_debt FROM debt;"},
    {"input": "Quel est le profit net de 2022 ?", "output": "SELECT SUM(net_profit) AS total_profit FROM profit WHERE strftime('%Y', date) = '2022';"},
    {"input": "Quelles sont les dépenses mensuelles pour chaque département en 2024 ?", "output": "SELECT department, strftime('%Y-%m', date) AS month, SUM(amount) AS monthly_expenses FROM expenses WHERE strftime('%Y', date) = '2024' GROUP BY department, month;"},
    {"input": "Quelles dépenses ont été faites en janvier 2024 ?", "output": "SELECT * FROM expenses WHERE strftime('%Y-%m', date) = '2024-01';"},
    {"input": "Combien le département IT a-t-il dépensé au T1 2023 ?", "output": "SELECT SUM(amount) AS it_expenses FROM expenses WHERE department = 'IT' AND strftime('%Y', date) = '2023' AND strftime('%m', date) IN ('01', '02', '03');"},
    {"input": "Quels départements ont un profit net supérieur à 50000 ?", "output": "SELECT department, SUM(net_profit) AS total_profit FROM profit GROUP BY department HAVING total_profit > 50000;"},
    {"input": "Quels employés ont un salaire supérieur à 5000 ?", "output": "SELECT name, salary FROM employees WHERE salary > 5000;"},
    {"input": "Quels sont les projets en cours ?", "output": "SELECT project_name FROM projects WHERE status = 'En cours';"},
    {"input": "Quel est le nombre total d'employés par département ?", "output": "SELECT department, COUNT(*) AS total_employees FROM employees GROUP BY department;"},
    {"input": "Quels produits ont été vendus en février 2024 ?", "output": "SELECT product_name FROM sales WHERE strftime('%Y-%m', sale_date) = '2024-02';"},
    {"input": "Quelle est la quantité totale vendue par produit ?", "output": "SELECT product_name, SUM(quantity) AS total_quantity FROM sales GROUP BY product_name;"},
    {"input": "Quels sont les clients avec un total d'achat supérieur à 1000 ?", "output": "SELECT customer_name, SUM(total_amount) AS total_purchase FROM orders GROUP BY customer_name HAVING total_purchase > 1000;"},
    {"input": "Combien de commandes ont été passées en 2024 ?", "output": "SELECT COUNT(*) AS total_orders FROM orders WHERE strftime('%Y', order_date) = '2024';"},
    {"input": "Quel est le stock actuel de chaque produit ?", "output": "SELECT product_name, stock_quantity FROM inventory;"},
    {"input": "Quels produits sont en rupture de stock ?", "output": "SELECT product_name FROM inventory WHERE stock_quantity = 0;"},
    {"input": "Quels fournisseurs ont livré des produits en 2023 ?", "output": "SELECT DISTINCT supplier_name FROM deliveries WHERE strftime('%Y', delivery_date) = '2023';"},
    {"input": "Quelle est la commande la plus récente ?", "output": "SELECT * FROM orders ORDER BY order_date DESC LIMIT 1;"},
    {"input": "Combien de clients ont passé au moins une commande ?", "output": "SELECT COUNT(DISTINCT customer_id) AS total_customers FROM orders;"},
    {"input": "Quel est le montant moyen des commandes ?", "output": "SELECT AVG(total_amount) AS average_order_value FROM orders;"},
    {"input": "Quels sont les produits les plus vendus ?", "output": "SELECT product_name, SUM(quantity) AS total_quantity FROM sales GROUP BY product_name ORDER BY total_quantity DESC;"},
    {"input": "Quels employés ont été embauchés en 2022 ?", "output": "SELECT name, hire_date FROM employees WHERE strftime('%Y', hire_date) = '2022';"},
    {"input": "Combien de jours de retard ont les projets ?", "output": "SELECT project_name, julianday(CURRENT_DATE) - julianday(deadline) AS days_late FROM projects WHERE status = 'En retard';"},
    {"input": "Quel est le salaire moyen par département ?", "output": "SELECT department, AVG(salary) AS average_salary FROM employees GROUP BY department;"},
    {"input": "Quels sont les 5 plus gros clients par montant d'achat ?", "output": "SELECT customer_name, SUM(total_amount) AS total_purchase FROM orders GROUP BY customer_name ORDER BY total_purchase DESC LIMIT 5;"},
    {"input": "Quels produits ont un prix unitaire supérieur à 50 ?", "output": "SELECT product_name, unit_price FROM products WHERE unit_price > 50;"},
    {"input": "Quels projets doivent se terminer en octobre 2024 ?", "output": "SELECT project_name FROM projects WHERE strftime('%Y-%m', deadline) = '2024-10';"},
    {"input": "What financial metrics does the dashboard display?", "output": "The dashboard shows metrics like cash flow, revenue, expenses, and profit."},
    {"input": "Can I generate financial reports?", "output": "Yes, you can generate reports in PDF, Excel, or CSV formats based on custom time periods and metrics."},
    {"input": "How can I see user information?", "output": "User information is available under 'Users' but is visible only to admins."},
    {"input": "Does Core Capital provide predictive financial insights?", "output": "Yes, it analyzes past expenses to provide insights and recommendations for future planning."},
    {"input": "How do I edit my profile picture?", "output": "Click on your profile picture in the navbar, and a modal will appear allowing you to change it."},
    {"input": "What is the purpose of the dashboard?", "output": "The dashboard provides an overview of your financial metrics and performance through charts and graphs."},
    {"input": "How can I customize my dashboard?", "output": "Click the 'Customize' button at the top right of the dashboard to modify the displayed metrics."},
    {"input": "Where can I find financial tutorials?", "output": "You can find tutorials in the 'Help' section or by asking the chatbot for specific topics."},
    {"input": "Can you guide me through creating a budget?", "output": "Sure! To create a budget, go to the 'Metrics' section and select 'Budget' to get started."},
    {"input": "How do I access my financial reports?", "output": "You can access your financial reports under the 'Reports' section of the dashboard."},
    {"input": "What are cash flow metrics?", "output": "Cash flow metrics measure the inflow and outflow of cash, indicating your business's liquidity."},
    {"input": "Can you help me understand profit margins?", "output": "Profit margins indicate how much profit a company makes for every dollar of sales. Higher margins mean more profitability."},
    {"input": "What is the best way to analyze expenses?", "output": "You can analyze expenses by reviewing the 'Expenses' table under the 'Metrics' section."},
    {"input": "How do I generate a cash flow report?", "output": "To generate a cash flow report, navigate to the 'Reports' section and select 'Cash Flow'."},
    {"input": "How can I improve my revenue?", "output": "Consider strategies like optimizing pricing, enhancing marketing efforts, and expanding product lines."},
    {"input": "What is an investment portfolio?", "output": "An investment portfolio is a collection of assets like stocks, bonds, and other investments held by an individual or institution."},
    {"input": "How do I track project budgets?", "output": "You can track project budgets by navigating to the 'Projects' section and reviewing individual project details."},
    {"input": "Can you provide a tutorial on financial forecasting?", "output": "Absolutely! Financial forecasting involves predicting future revenues and expenses based on historical data."},
    {"input": "How do I edit a project?", "output": "To edit a project, go to the 'Projects' section, select the project, and click 'Edit'."},
    {"input": "What metrics should I focus on for financial health?", "output": "Key metrics include cash flow, profit margins, debt levels, and return on investment."},
    {"input": "How can I change my language settings?", "output": "Click on the language icon in the navbar, but please note that this functionality is not currently operational."},
    {"input": "Can you explain how to reduce debt?", "output": "To reduce debt, consider increasing revenue, cutting unnecessary expenses, and allocating extra funds toward debt repayment."},
    {"input": "What is an expense report?", "output": "An expense report is a document used to track and summarize business expenses incurred during a specific period."},
    {"input": "How do I set financial goals?", "output": "You can set financial goals by assessing your current financial situation and outlining desired outcomes."},
    {"input": "How can I get help with investment strategies?", "output": "You can find investment strategy resources in the Help section or ask for specific guidance."},
    {"input": "What are the key components of a financial plan?", "output": "A financial plan typically includes budgeting, cash flow analysis, investment strategies, and risk management."},
    {"input": "How can I learn about financial ratios?", "output": "Financial ratios are essential for analyzing a company's performance. You can learn more in our tutorials section."},
    {"input": "What is the purpose of financial audits?", "output": "Financial audits assess the accuracy of financial statements and ensure compliance with regulations."},
    {"input": "How do I improve operational efficiency?", "output": "You can improve operational efficiency by streamlining processes, reducing waste, and optimizing resource allocation."},
    {"input": "Can you show me how to conduct a break-even analysis?", "output": "A break-even analysis determines when your business will be able to cover all its expenses. You can find tutorials on this in the Help section."},
    {"input": "What should I do if I'm experiencing cash flow issues?", "output": "Consider cutting unnecessary expenses, improving collections, and renegotiating payment terms with suppliers."},
    {"input": "How do I understand my financial statements?", "output": "Financial statements provide insight into your business's performance. You can find tutorials on interpreting these in the Help section."},
    {"input": "How can I access customer financial data?", "output": "Customer financial data is available under 'Users,' accessible only to admins."},
    {"input": "Can you help me with tax planning?", "output": "Tax planning involves strategies to minimize tax liabilities. You can find more information in our financial tutorials."},
    {"input": "How often should I review my financial metrics?", "output": "Regular reviews, at least monthly, help you stay informed about your business's financial health."},
    {"input": "What is financial modeling?", "output": "Financial modeling involves creating a representation of a business's financial performance under various scenarios."},
    {"input": "What are the benefits of budgeting?", "output": "Budgeting helps control spending, manage cash flow, and plan for future expenses."},
    {"input": "How do I analyze my investment performance?", "output": "You can analyze investment performance by comparing returns against benchmarks and assessing risk levels."},
    {"input": "What is the difference between gross and net profit?", "output": "Gross profit is revenue minus the cost of goods sold, while net profit is gross profit minus all other expenses."},
    {"input": "How can I reduce operational costs?", "output": "Identify areas of waste, negotiate with suppliers, and consider outsourcing non-core activities."},
    {"input": "How do I conduct a market analysis?", "output": "A market analysis involves researching industry trends, competitor performance, and target customer preferences."},
    {"input": "What should I include in a financial presentation?", "output": "Include key metrics, trends, forecasts, and visual aids to effectively communicate financial information."},
    {"input": "How can I improve cash flow?", "output": "Enhance cash flow by accelerating receivables, extending payables, and reducing unnecessary expenses."},
    {"input": "What is working capital?", "output": "Working capital is the difference between current assets and current liabilities, indicating short-term financial health."},
    {"input": "How can I assess investment risks?", "output": "Assess investment risks by analyzing market trends, historical performance, and economic indicators."},
    {"input": "What are fixed and variable costs?", "output": "Fixed costs remain constant regardless of production levels, while variable costs fluctuate with production volume."},
    {"input": "How do I create a financial forecast?", "output": "Gather historical data, analyze trends, and project future revenues and expenses based on informed assumptions."},
    {"input": "What is a cash flow statement?", "output": "A cash flow statement summarizes cash inflows and outflows over a specific period, highlighting liquidity."},
    {"input": "How can I improve my credit score?", "output": "Improve your credit score by paying bills on time, reducing debt, and maintaining low credit utilization."},
    {"input": "What is a financial KPI?", "output": "A financial KPI (Key Performance Indicator) measures a company's financial performance against objectives."},
    {"input": "How can I understand financial ratios better?", "output": "Financial ratios provide insights into efficiency, profitability, and liquidity. You can find tutorials in our Help section."},
    {"input": "What is a return on investment (ROI)?", "output": "ROI measures the gain or loss generated relative to the investment cost, expressed as a percentage."},
    {"input": "How do I analyze my company's profitability?", "output": "Analyze profitability by reviewing profit margins, return on equity, and return on assets."},
    {"input": "Can you explain the significance of liquidity ratios?", "output": "Liquidity ratios measure a company's ability to meet short-term obligations, indicating financial health."},
    {"input": "How can I set up automatic payments?", "output": "You can set up automatic payments through your bank or accounting software for regular expenses."},
    {"input": "What is a business credit line?", "output": "A business credit line is a flexible loan option allowing businesses to borrow up to a limit as needed."},
    {"input": "How do I calculate my break-even point?", "output": "The break-even point is calculated by dividing total fixed costs by the contribution margin per unit."},
    {"input": "Can I track my investments on Core Capital?", "output": "Yes, you can track investments by navigating to the 'Investments' section of the dashboard."},
    {"input": "What are the benefits of using Core Capital?", "output": "Core Capital offers comprehensive financial analysis tools, reports, and user-friendly dashboards for better financial management."},
    {"input": "How can I find financial advisors?", "output": "You can find financial advisors through online directories, financial institutions, or referrals from trusted sources."},
    {"input": "What is an income statement?", "output": "An income statement summarizes a company's revenues, expenses, and profits over a specific period."},
    {"input": "How can I assess my financial risks?", "output": "Assess financial risks by analyzing market volatility, operational challenges, and regulatory changes."},
    {"input": "How do I keep track of business expenses?", "output": "You can track expenses using accounting software or by maintaining organized spreadsheets."},
    {"input": "What are the benefits of a diversified investment portfolio?", "output": "Diversification reduces risk by spreading investments across various asset classes."},
    {"input": "How do I calculate net income?", "output": "Net income is calculated by subtracting total expenses from total revenues."},
    {"input": "What is the importance of financial literacy?", "output": "Financial literacy helps individuals and businesses make informed decisions regarding budgeting, investing, and saving."},
    {"input": "Can you provide tips for effective budgeting?", "output": "Effective budgeting involves setting clear goals, tracking spending, and regularly reviewing financial performance."},
    {"input": "How do I implement cost-saving strategies?", "output": "Implement cost-saving strategies by analyzing expenses, renegotiating contracts, and seeking efficiencies."},
    {"input": "What are the advantages of using technology in finance?", "output": "Technology enhances efficiency, accuracy, and accessibility of financial data and analysis."},
    {"input": "How can I measure customer profitability?", "output": "Customer profitability can be measured by analyzing revenue generated versus the cost of acquiring and servicing customers."},
    {"input": "What is a financial advisor's role?", "output": "A financial advisor helps clients manage investments, plan for retirement, and make informed financial decisions."},
    {"input": "How do I prepare for a financial audit?", "output": "Prepare for a financial audit by organizing financial documents, ensuring accuracy, and being transparent."},
    {"input": "What is a balance sheet?", "output": "A balance sheet is a financial statement that presents a company's assets, liabilities, and equity at a specific point in time."},
    {"input": "Can you help me understand asset allocation?", "output": "Asset allocation is the strategy of distributing investments across different asset classes to manage risk."},
    {"input": "How do I analyze competitor financial performance?", "output": "Analyze competitor financial performance by reviewing their financial statements, market share, and profitability ratios."},
    {"input": "What is financial leverage?", "output": "Financial leverage refers to using borrowed funds to increase the potential return on investment."},
    {"input": "How can I find funding for my business?", "output": "Consider exploring options like loans, investors, grants, and crowdfunding to find funding."},
    {"input": "What is a capital expenditure?", "output": "A capital expenditure (CapEx) is an investment in long-term assets or improvements for future benefits."},
    {"input": "How do I create a financial dashboard?", "output": "Create a financial dashboard by selecting key metrics and using visualization tools to present data effectively."},
    {"input": "What should I consider when planning for retirement?", "output": "Consider your retirement goals, expected expenses, savings rates, and investment strategies."},
    {"input": "How can I improve employee productivity?", "output": "Improve productivity by fostering a positive work environment, providing training, and recognizing employee contributions."},
    {"input": "What are the risks of investing?", "output": "Investment risks include market volatility, economic downturns, interest rate changes, and company performance."},
    {"input": "How do I analyze business performance?", "output": "Analyze business performance by reviewing financial statements, KPIs, and comparing against industry benchmarks."},
    {"input": "What is a financial crisis?", "output": "A financial crisis refers to a situation where financial institutions or assets suddenly lose a significant portion of their value."},
    {"input": "How can I improve my negotiation skills?", "output": "Improve negotiation skills by practicing, understanding your needs, and being open to compromise."},
    {"input": "What are the common financial pitfalls?", "output": "Common pitfalls include overspending, neglecting savings, and failing to plan for taxes."},
    {"input": "How can I enhance my personal financial literacy?", "output": "Enhance financial literacy by reading books, attending workshops, and utilizing online resources."},
    {"input": "What is a financial risk assessment?", "output": "A financial risk assessment identifies potential risks that could impact a business's financial performance."},
    {"input": "How can I set a realistic financial budget?", "output": "Set a realistic budget by analyzing past expenses, setting clear goals, and regularly reviewing financial performance."},
    {"input": "What are the different types of investments?", "output": "Types of investments include stocks, bonds, mutual funds, ETFs, real estate, and commodities."},
    {"input": "How do I evaluate investment opportunities?", "output": "Evaluate opportunities by analyzing potential returns, risks, market trends, and the investment's alignment with your goals."},
    {"input": "What is cash management?", "output": "Cash management involves optimizing the collection, handling, and usage of cash to meet business needs."},
    {"input": "How do I establish financial controls?", "output": "Establish controls by implementing policies, procedures, and checks to ensure accurate financial reporting."},
    {"input": "What is the significance of shareholder equity?", "output": "Shareholder equity represents the residual interest in a company's assets after deducting liabilities."},
    {"input": "How can I build a financial safety net?", "output": "Build a safety net by saving an emergency fund, reducing debt, and ensuring proper insurance coverage."},
    {"input": "What is the role of a CFO?", "output": "A CFO (Chief Financial Officer) oversees financial planning, risk management, and financial reporting for a company."},
    {"input": "How do I measure financial success?", "output": "Measure success by tracking financial metrics like revenue growth, profitability, and return on investment."},
    {"input": "What are the tax implications of investments?", "output": "Investment tax implications can include capital gains tax, dividend tax, and interest income tax."},
    {"input": "How can I effectively manage cash flow?", "output": "Manage cash flow by closely monitoring receivables, payables, and implementing cash flow forecasts."},
    {"input": "What is the difference between saving and investing?", "output": "Saving is setting aside money for short-term goals, while investing involves using money to generate returns over the long term."},
    {"input": "How can I improve my negotiation skills?", "output": "Improve negotiation skills by practicing active listening, understanding interests, and seeking win-win outcomes."},
    {"input": "What is the importance of regular financial reviews?", "output": "Regular financial reviews help identify trends, adjust budgets, and make informed decisions."},
    {"input": "How do I ensure compliance with financial regulations?", "output": "Ensure compliance by staying updated on regulations, implementing necessary procedures, and conducting audits."},
    {"input": "What is the purpose of a financial audit?", "output": "A financial audit assesses the accuracy of financial records and compliance with applicable laws."},
    {"input": "How can I manage financial stress?", "output": "Manage financial stress by creating a budget, setting financial goals, and seeking professional advice."},
    {"input": "What are the benefits of early retirement planning?", "output": "Early planning allows for more savings, investment growth, and a better understanding of retirement needs."},
    {"input": "How do I navigate financial markets?", "output": "Navigate markets by staying informed about economic trends, market news, and investment opportunities."},
    {"input": "What are the challenges of managing finances?", "output": "Challenges include budgeting, unexpected expenses, market volatility, and financial literacy."},
    {"input": "How can I build a strong financial foundation?", "output": "Build a foundation by saving regularly, investing wisely, and maintaining a budget."},
    {"input": "What should I include in a financial plan?", "output": "Include goals, budgets, investment strategies, retirement plans, and risk management."},
    {"input": "How can I enhance my financial resilience?", "output": "Enhance resilience by diversifying income sources, saving for emergencies, and investing wisely."},
]

prompt =""" Your name is B4, and you are a helpful assistant for Core Capital. You are trained to generate detailed financial reports, answer user queries, and assist with platform features based on the provided context. When interacting with users, your responses should be accurate and aligned with the data available in the system. 

Core Capital is a platform focused on corporate financial health, offering tools for real-time financial monitoring, expense tracking, automated reporting, and predictive insights. 

Tasks you can handle:
1. Answer questions about financial metrics like revenue, expenses, and profit.
2. Generate financial reports based on user-defined parameters (time periods, formats, metrics).
3. Provide recommendations based on past expense data to help users plan their budgets.
4. Assist users in navigating the platform (dashboard, users, reports, and settings).
5. Search for information in the user database and provide quick responses.
6. Predict future trends based on past financial data and display metrics in real time.

You are available via a chatbot interface located in the bottom-right corner of the platform. Remember to keep your answers concise and avoid unnecessary details unless requested by the user. Your role is to enhance the user experience and make financial management easy and efficient.
"""

In [5]:
from transformers import AutoTokenizer

model_path = "fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Tokenisation des données d'entraînement
def preprocess_function(data):
    return tokenizer(
        data["input"], text_target=data["output"], truncation=True, padding="max_length", max_length=100
    )
# Appliquez la tokenisation sur vos données
tokenized_data = [preprocess_function(item) for item in train_data]


In [6]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# Charger le modèle pré-entraîné
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    per_device_train_batch_size=1,    # Taille de batch
    num_train_epochs=1,
    report_to="none",
    gradient_accumulation_steps=2,    # Réduire le nombre de pas d'accumulation de gradients             
    fp16=True 
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from torch.utils.data import Dataset

# Créez une classe Dataset personnalisée pour vos données
class CustomDataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": self.data[idx]["input_ids"],
            "labels": self.data[idx]["input_ids"]
        }
# Charger les données dans le dataset
train_dataset = CustomDataset(tokenized_data)


In [8]:
# Créez le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

In [ ]:
import torch
import gc 
gc.collect()
torch.cuda.empty_cache()

In [ ]:
input_text = "quelle sont les revenue de ce mois-ci ?"
input_tokens = tokenizer(input_text, return_tensors="pt").to("cuda")

output = model.generate(**input_tokens, max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))